<a href="https://colab.research.google.com/github/JKBB2303/Ignite/blob/main/Spotify_Song_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Setup

In [ ]:
# Install specific libraries not included in Colab by default
!pip install transformers

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity


Note: for this line to work, you need to have privously uploated the dataset from kaggle into google collab - under content.

kaggle dataset link:
https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset?resource=download

In [ ]:
# Importing Spotify dataset
df = pd.read_csv('/content/dataset.csv')

<ipython-input-16-150eaca45217>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/dataset.csv')


In [ ]:
df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [ ]:
[i for i in df['Unnamed: 0'] if (not isinstance(i, int)) or i > df.shape[0]]

[]

In [ ]:
df['Unnamed: 0'].iloc[-1]

113999

In [ ]:
df.shape

(114000, 21)

In [ ]:
for i in range(0, df.shape[0]):
  if i != int(df['Unnamed: 0'].iloc[i]):
    print(i)

Note: unnamed here is basically another index count for unique tracks. Let's rename it for simplicity to track_index
WE WILL ONLY USE EITHER THE UNNAMED: 0 COLUMN OR THE TRACK ID COLUMN

In [ ]:
df.rename(columns={'Unnamed: 0': 'track_index'}, inplace=True)

In [ ]:
df.head()

,track_index,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [ ]:
df.columns

Index(['track_index', 'track_id', 'artists', 'album_name', 'track_name',
       'popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'track_genre'],
      dtype='object')

In [ ]:
# datatype description
df.dtypes

track_index           int64
track_id             object
artists              object
album_name           object
track_name           object
popularity            int64
duration_ms           int64
explicit               bool
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
track_genre          object
dtype: object

In [ ]:
[x for x in df.artists]     # note that sometimes we have double artists separated by commas

['Gen Hoshino',
 'Ben Woodward',
 'Ingrid Michaelson;ZAYN',
 'Kina Grannis',
 'Chord Overstreet',
 'Tyrone Wells',
 'A Great Big World;Christina Aguilera',
 'Jason Mraz',
 'Jason Mraz;Colbie Caillat',
 'Ross Copperman',
 'Zack Tabudlo',
 'Jason Mraz',
 'Dan Berk',
 'Anna Hamilton',
 'Chord Overstreet;Deepend',
 'Landon Pigg',
 'Andrew Foy;Renee Foy',
 'Andrew Foy;Renee Foy',
 'Jason Mraz;Colbie Caillat',
 'Boyce Avenue;Bea Miller',
 'Jason Mraz',
 'Boyce Avenue;Jennel Garcia',
 'A Great Big World;Christina Aguilera',
 'Jason Mraz',
 'Jason Mraz',
 'Jason Mraz',
 'Jason Mraz',
 'Jason Mraz',
 'Jason Mraz',
 'Jason Mraz',
 'Jason Mraz',
 'Jason Mraz',
 'Chord Overstreet',
 'Brandi Carlile;Sam Smith',
 'Brandi Carlile;Sam Smith',
 'Brandi Carlile;Sam Smith',
 'KT Tunstall',
 'Brandi Carlile',
 'Brandi Carlile',
 'KT Tunstall',
 'Eddie Vedder',
 'Brandi Carlile',
 'Brandi Carlile;Lucius',
 'Brandi Carlile;Lucius',
 'Brandi Carlile',
 'Brandi Carlile;Lucius',
 'Brandi Carlile;Lucius',
 'Bra

In [ ]:
[x for x in df.artists if ';' in str(x)]     # You can see those cases here

['Ingrid Michaelson;ZAYN',
 'A Great Big World;Christina Aguilera',
 'Jason Mraz;Colbie Caillat',
 'Chord Overstreet;Deepend',
 'Andrew Foy;Renee Foy',
 'Andrew Foy;Renee Foy',
 'Jason Mraz;Colbie Caillat',
 'Boyce Avenue;Bea Miller',
 'Boyce Avenue;Jennel Garcia',
 'A Great Big World;Christina Aguilera',
 'Brandi Carlile;Sam Smith',
 'Brandi Carlile;Sam Smith',
 'Brandi Carlile;Sam Smith',
 'Brandi Carlile;Lucius',
 'Brandi Carlile;Lucius',
 'Brandi Carlile;Lucius',
 'Brandi Carlile;Lucius',
 'Brandi Carlile;Lucius',
 'Brandi Carlile;Lucius',
 'Brandi Carlile;Lucius',
 'Boyce Avenue;Bea Miller',
 'Eddie Vedder;Nusrat Fateh Ali Khan',
 'Boyce Avenue;Fifth Harmony',
 'Gabrielle Aplin;JP Cooper',
 'Andrew Foy;Renee Foy',
 'Ray LaMontagne;Sierra Ferrell',
 'Susie Suh;Robot Koch',
 'Tyler Ward;Lindsey Stirling;Kina Grannis',
 'Eden Elf;Ren Avel',
 'Boyce Avenue;Rachel Grae',
 'A Great Big World;Christina Aguilera',
 'Zack Tabudlo;Yonnyboii',
 'Andrew Foy;Renee Foy',
 'Tyler Ward;Karis;Ray 

In [ ]:
# all genres are singular however. No lists: each song/track has a single genre
df['track_genre'].unique()

array(['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient',
       'anime', 'black-metal', 'bluegrass', 'blues', 'brazil',
       'breakbeat', 'british', 'cantopop', 'chicago-house', 'children',
       'chill', 'classical', 'club', 'comedy', 'country', 'dance',
       'dancehall', 'death-metal', 'deep-house', 'detroit-techno',
       'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm',
       'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk',
       'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove',
       'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle',
       'heavy-metal', 'hip-hop', 'honky-tonk', 'house', 'idm', 'indian',
       'indie-pop', 'indie', 'industrial', 'iranian', 'j-dance', 'j-idol',
       'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino',
       'malay', 'mandopop', 'metal', 'metalcore', 'minimal-techno', 'mpb',
       'new-age', 'opera', 'pagode', 'party', 'piano', 'pop-film', 'pop',
       'pow

In [ ]:
df.artists.unique()

array(['Gen Hoshino', 'Ben Woodward', 'Ingrid Michaelson;ZAYN', ...,
       'Cuencos Tibetanos Sonidos Relajantes',
       'Bryan & Katie Torwalt;Brock Human', 'Jesus Culture'], dtype=object)

In [ ]:
[x for x in df.artists if not isinstance(x, str)]   # 1 nan value exists in artists

[nan]

In [ ]:
# convert artists into lists of artists
# print(df['artists'])
# def artists_to_list(df):
#   # updates artists col from a string separated by ';' into a list of artists.

#   # iterate over the artist col and when you find the semi-colon, do the split

#   # also returns the new df
#   df['artists'] = df['artists'].apply(lambda x: x.split(";"))
#   return df
# artists_to_list(df)
def artist_to_list(s):
  if isinstance(s, str):
    return s.split(";")


In [ ]:
df['artists'] = df['artists'].apply(lambda x: artist_to_list(x))

# Ignore this

In [ ]:
# Importing Spotify dataset
# df = pd.read_csv('/content/sample_data/dataset.csv')

# # Try this: downloading from url using wget - probably need more code to fix this
# !pip install wget
# import wget
# df = wget.download("https://www.kaggle.com/datasets/maharshipandya/-spotify-tracks-dataset.download")
# # !wget "https://www.kaggle.com/maharshipandya/-spotify-tracks-dataset/download" -O dataset.zip

# kaggle API command: kaggle datasets download -d maharshipandya/-spotify-tracks-dataset

In [ ]:
# # see how df looks
# df

'-spotify-tracks-dataset'

# Back to our code

Cleaning our data

In [ ]:
df.shape

(114000, 21)

In [ ]:
df[df.isnull().any(axis=1)]   # there is exactly 1 row that has NaN values

,track_index,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
65900,65900,1kR4gIb7nGxHPI3D2ifs59,None,NaN,NaN,0,0,False,0.501,0.583,...,-9.46,0,0.0605,0.69,0.00396,0.0747,0.734,138.391,4,k-pop


In [ ]:
df = df.dropna()  # let's drop that row from our dataframe

In [ ]:
df.shape

(113999, 21)

# TFIDF Vecotrization
This process is done for: track_genre, artists
Everything is first done for genres, then also done for artists

In [ ]:
[x for x in df['track_genre'] if not isinstance(x, str)]    # all genre values are strings

[]

In [ ]:
# convert all genres to lists (these would be lists of one)
# genre_lists = df['track_genre'].apply(lambda x: x.split(";"))

In [ ]:
# genre_lists

0            [acoustic]
1            [acoustic]
2            [acoustic]
3            [acoustic]
4            [acoustic]
              ...      
113995    [world-music]
113996    [world-music]
113997    [world-music]
113998    [world-music]
113999    [world-music]
Name: track_genre, Length: 113999, dtype: object

In [ ]:
# # df['artists'] is already stored as a list, so just make a copy of it
# artist_lists = df[['artists']].copy()
# artist_lists

,artists
0,[Gen Hoshino]
1,[Ben Woodward]
2,"[Ingrid Michaelson, ZAYN]"
3,[Kina Grannis]
4,[Chord Overstreet]
...,...
113995,[Rainy Lullaby]
113996,[Rainy Lullaby]
113997,[Cesária Evora]
113998,[Michael W. Smith]


In [ ]:
df['track_genre'] = df['track_genre'].apply(lambda x: [x])  # adds a []
# df['track_genre'] = df['track_genre'].apply(lambda x: x[0]) # removes a []
df['track_genre']

0            [acoustic]
1            [acoustic]
2            [acoustic]
3            [acoustic]
4            [acoustic]
              ...      
113995    [world-music]
113996    [world-music]
113997    [world-music]
113998    [world-music]
113999    [world-music]
Name: track_genre, Length: 113999, dtype: object

In [ ]:
# TFIDF for genres
tfidf = TfidfVectorizer()
genres_tfidf_matrix = tfidf.fit_transform(df['track_genre'].apply(lambda x: " ".join(x)))
genre_df = pd.DataFrame(genres_tfidf_matrix.toarray())
genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]   # this makes the column heads reflect the genres that they represent
genre_df.reset_index(drop = True, inplace=True)
# df = pd.get_dummies(df, columns=['track_genre'])

In [ ]:
# genre df        - note: we don't put this back into the main dataframe
genre_df

,genre|acoustic,genre|afrobeat,genre|age,genre|alt,genre|alternative,genre|ambient,genre|and,genre|anime,genre|bass,genre|black,...,genre|swedish,genre|synth,genre|tango,genre|techno,genre|tonk,genre|trance,genre|trip,genre|tunes,genre|turkish,genre|world
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107
113995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107
113996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107
113997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.707107


In [ ]:
# The process for artists would look slightly different as we are passing in a list with multiple elements
# Since artist names may have spaces, we must define a custom tokenizer that uses ';' instead
# We then pass this custom_tokenizer to our tfidf vectorizer
def custom_tokenizer(text):
    return text.split(';')

custom_tfidf = TfidfVectorizer(tokenizer=custom_tokenizer)

In [ ]:
# Now let's do TFIDF vectorization for artists
artists_tfidf_matrix = custom_tfidf.fit_transform(df['artists'].apply(lambda x: ";".join(x)))
artists_df = pd.DataFrame(artists_tfidf_matrix.toarray())
artists_df.columns = ['artist' + "|" + i for i in custom_tfidf.get_feature_names_out()]
artists_df.reset_index(drop = True, inplace=True)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
# FINALLY!
artists_df

,artist|!nvite,"artist|""cats"" 1983 broadway cast","artist|""little"" jimmie dickens","artist|""pepo"" san martín","artist|""puppy dog pals"" cast","artist|""sax"" maldita vecindad","artist|""the phantom of the opera"" 1990 german cast","artist|""weird al"" yankovic",artist|#kids,artist|$affie,...,artist|鶴見知利子（cv.早見沙織）,artist|麗莎,artist|黃仲崑,artist|黃妃,artist|黃小琥,artist|黃德斌,artist|黃敏華,artist|黄霄雲,artist|齋藤摩羅衛門,artist|龍藏ryuzo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
artists_df.shape

(113999, 29812)

In [ ]:
# It seems like our code works as we wanted
print(artists_df.columns[artists_df.iloc[0] == 1])
print(artists_df.columns[artists_df.iloc[1] == 1])
print(artists_df.columns[artists_df.iloc[2] != 0])
print()
artists_df[['artist|ingrid michaelson', 'artist|zayn']].iloc[2]

Index(['artist|gen hoshino'], dtype='object')
Index(['artist|ben woodward'], dtype='object')
Index(['artist|ingrid michaelson', 'artist|zayn'], dtype='object')



artist|ingrid michaelson    0.707107
artist|zayn                 0.707107
Name: 2, dtype: float64

# One-hot Encoding

In [ ]:
df['key']

0         1
1         1
2         0
3         0
4         2
         ..
113995    5
113996    0
113997    0
113998    7
113999    1
Name: key, Length: 113999, dtype: int64

In [ ]:
# code for one-hot encoding here     - add more col names here as needed
# use it on: 'mode', 'key'

# for column in ('mode', 'key'):

def col_ohe(df, column):
  term_freq_col = pd.get_dummies(df[column])
  feature_names = term_freq_col.columns
  term_freq_col.columns = [column + "|" + str(i) for i in feature_names]
  term_freq_col.reset_index(drop=True, inplace=True)
  return term_freq_col


one_hot_mode = col_ohe(df, 'mode')
one_hot_key = col_ohe(df, 'key')

In [ ]:
one_hot_key

,key|0,key|1,key|2,key|3,key|4,key|5,key|6,key|7,key|8,key|9,key|10,key|11
0,False,True,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,False,False
4,False,False,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
113994,False,False,False,False,False,True,False,False,False,False,False,False
113995,True,False,False,False,False,False,False,False,False,False,False,False
113996,True,False,False,False,False,False,False,False,False,False,False,False
113997,False,False,False,False,False,False,False,True,False,False,False,False


# Drop Unwanted Columns
You can comment or uncomment lines from this to remove different columns from the dataset, if you don't want them
BUT: be sure to edit the code below if you change this part!!!

In [ ]:
# Optional line: these lines, uncommented, would remove popularity and explicitness.
# We'll remove the popularity score from our dataset as it quickly changes overtime, and becomes irrelevant
df = df.drop('popularity', axis = 1)
# df = df.drop('explicit', axis = 1)

In [ ]:
df.head()

,track_index,track_id,artists,album_name,track_name,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,[Gen Hoshino],Comedy,Comedy,230666,0.0,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,[acoustic]
1,1,4qPNDBW1i3p13qLCt0Ki3A,[Ben Woodward],Ghost (Acoustic),Ghost - Acoustic,149610,0.0,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,[acoustic]
2,2,1iJBSr7s7jYXzM8EGcbK5b,"[Ingrid Michaelson, ZAYN]",To Begin Again,To Begin Again,210826,0.0,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,[acoustic]
3,3,6lfxq3CG4xtTiEg7opyCyx,[Kina Grannis],Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,201933,0.0,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,[acoustic]
4,4,5vjLSffimiIP26QG5WcN2K,[Chord Overstreet],Hold On,Hold On,198853,0.0,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,[acoustic]


# Normalization

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# note: mode and key are not in this (instead one hot encoded). Should we include time_signature and tempo here or with them?
# we may have dropped popularity or explicitness from the dataset in the previous section
# MAKE SURE: if popularity was/wasn't dropped, it shouldn't/should be here as the first item
# Assuming 'numerical_features' is a list of your numerical column names that only need to be potentially rescaled
numerical_features = ['explicit', 'duration_ms', 'danceability', 'energy',
                      'loudness', 'speechiness', 'acousticness',
                      'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']

scaler = MinMaxScaler()
# df[numerical_features] = scaler.fit_transform(df[numerical_features])

audio = df[numerical_features].reset_index(drop= True)
audio_scaled = pd.DataFrame(scaler.fit_transform(audio), columns = audio.columns)


How to deal with 'explicit'
Unlike our other values, this column is made up of booleans. We can either convert it to floats of 0 and 1, or drop it completely
We will go with the first option. But first let's check that it would work okay before alterring the main dataframe

In [ ]:
# This would convert explicit values from True and False to 0 and 1
# Done this on a copy so as not to change the actual dataframe
df_copy = df.copy()
df_copy[['explicit']] = scaler.fit_transform(df_copy[['explicit']])
df_copy.head()

,track_index,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,[Gen Hoshino],Comedy,Comedy,73,230666,0.0,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,[acoustic]
1,1,4qPNDBW1i3p13qLCt0Ki3A,[Ben Woodward],Ghost (Acoustic),Ghost - Acoustic,55,149610,0.0,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,[acoustic]
2,2,1iJBSr7s7jYXzM8EGcbK5b,"[Ingrid Michaelson, ZAYN]",To Begin Again,To Begin Again,57,210826,0.0,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,[acoustic]
3,3,6lfxq3CG4xtTiEg7opyCyx,[Kina Grannis],Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,0.0,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,[acoustic]
4,4,5vjLSffimiIP26QG5WcN2K,[Chord Overstreet],Hold On,Hold On,82,198853,0.0,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,[acoustic]


In [ ]:
df.head()

,track_index,track_id,artists,album_name,track_name,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,[Gen Hoshino],Comedy,Comedy,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,[acoustic]
1,1,4qPNDBW1i3p13qLCt0Ki3A,[Ben Woodward],Ghost (Acoustic),Ghost - Acoustic,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,[acoustic]
2,2,1iJBSr7s7jYXzM8EGcbK5b,"[Ingrid Michaelson, ZAYN]",To Begin Again,To Begin Again,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,[acoustic]
3,3,6lfxq3CG4xtTiEg7opyCyx,[Kina Grannis],Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,[acoustic]
4,4,5vjLSffimiIP26QG5WcN2K,[Chord Overstreet],Hold On,Hold On,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,[acoustic]


In [ ]:
# [x for x in df['explicit'] if int(x) != df_copy['explicit']]

# This prints nothing meaning that explicit was correctly converted to numerical
for i in range(0, df.shape[0]):
  original_e = df['explicit'].iloc[i]
  new_e = df_copy['explicit'].iloc[i]
  if int(original_e) != new_e:
    print(i, end='    ')
    print(original_e, end='    ')
    print(new_e)

Let's convert 'explicit' into a scalar

In [ ]:
# Added 'explicit' to the numerical_features above, so it would be scaled with everything else
# df[['explicit']] = scaler.fit_transform(df[['explicit']])

In [ ]:
df.head()

,track_index,track_id,artists,album_name,track_name,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,[Gen Hoshino],Comedy,Comedy,230666,0.0,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,[acoustic]
1,1,4qPNDBW1i3p13qLCt0Ki3A,[Ben Woodward],Ghost (Acoustic),Ghost - Acoustic,149610,0.0,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,[acoustic]
2,2,1iJBSr7s7jYXzM8EGcbK5b,"[Ingrid Michaelson, ZAYN]",To Begin Again,To Begin Again,210826,0.0,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,[acoustic]
3,3,6lfxq3CG4xtTiEg7opyCyx,[Kina Grannis],Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,201933,0.0,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,[acoustic]
4,4,5vjLSffimiIP26QG5WcN2K,[Chord Overstreet],Hold On,Hold On,198853,0.0,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,[acoustic]


# Miscellaneous
skip while running the main code


Contains:
- list of unique artists
- backend input format
- backend output format

In [ ]:
# Get list of all unique artists
# df['artists']
# sum(df['artists'], [])    # This works but take very long - instead, let's use explode()
# if we wanted to use this, we'd apply the set() function on the sum, and then turn that set into a list

['Gen Hoshino',
 'Ben Woodward',
 'Ingrid Michaelson',
 'ZAYN',
 'Kina Grannis',
 'Chord Overstreet',
 'Tyrone Wells',
 'A Great Big World',
 'Christina Aguilera',
 'Jason Mraz',
 'Jason Mraz',
 'Colbie Caillat',
 'Ross Copperman',
 'Zack Tabudlo',
 'Jason Mraz',
 'Dan Berk',
 'Anna Hamilton',
 'Chord Overstreet',
 'Deepend',
 'Landon Pigg',
 'Andrew Foy',
 'Renee Foy',
 'Andrew Foy',
 'Renee Foy',
 'Jason Mraz',
 'Colbie Caillat',
 'Boyce Avenue',
 'Bea Miller',
 'Jason Mraz',
 'Boyce Avenue',
 'Jennel Garcia',
 'A Great Big World',
 'Christina Aguilera',
 'Jason Mraz',
 'Jason Mraz',
 'Jason Mraz',
 'Jason Mraz',
 'Jason Mraz',
 'Jason Mraz',
 'Jason Mraz',
 'Jason Mraz',
 'Jason Mraz',
 'Chord Overstreet',
 'Brandi Carlile',
 'Sam Smith',
 'Brandi Carlile',
 'Sam Smith',
 'Brandi Carlile',
 'Sam Smith',
 'KT Tunstall',
 'Brandi Carlile',
 'Brandi Carlile',
 'KT Tunstall',
 'Eddie Vedder',
 'Brandi Carlile',
 'Brandi Carlile',
 'Lucius',
 'Brandi Carlile',
 'Lucius',
 'Brandi Carlile

In [ ]:
artists_copy = df[['artists']].copy()
artists_copy.head()

,artists
0,[Gen Hoshino]
1,[Ben Woodward]
2,"[Ingrid Michaelson, ZAYN]"
3,[Kina Grannis]
4,[Chord Overstreet]


In [ ]:
artists_copy = artists_copy.explode('artists')    # This separates list items into their own rows (look at how row 2 from before turned into 2 rows here)
artists_copy.head()       # We don't need to worry about the multiple indexes

,artists
0,Gen Hoshino
1,Ben Woodward
2,Ingrid Michaelson
2,ZAYN
3,Kina Grannis


In [ ]:
unique_artists = artists_copy['artists'].unique()
print(artists_copy.shape[0])
print(len(unique_artists))
unique_artists

158292
29859


array(['Gen Hoshino', 'Ben Woodward', 'Ingrid Michaelson', ...,
       'Molly Skaggs', 'Cuencos Tibetanos Sonidos Relajantes',
       'Brock Human'], dtype=object)

In [ ]:
# THE LIST OF UNIQUE ARTISTS
unique_artists = list(unique_artists)
unique_artists

['Gen Hoshino',
 'Ben Woodward',
 'Ingrid Michaelson',
 'ZAYN',
 'Kina Grannis',
 'Chord Overstreet',
 'Tyrone Wells',
 'A Great Big World',
 'Christina Aguilera',
 'Jason Mraz',
 'Colbie Caillat',
 'Ross Copperman',
 'Zack Tabudlo',
 'Dan Berk',
 'Anna Hamilton',
 'Deepend',
 'Landon Pigg',
 'Andrew Foy',
 'Renee Foy',
 'Boyce Avenue',
 'Bea Miller',
 'Jennel Garcia',
 'Brandi Carlile',
 'Sam Smith',
 'KT Tunstall',
 'Eddie Vedder',
 'Lucius',
 'Highland Peak',
 'Motohiro Hata',
 'Andrew Belle',
 'Ron Pope',
 'Adam Christopher',
 'Aron Wright',
 'Sara Bareilles',
 'Kurt Cobain',
 'Tim Halperin',
 'Canyon City',
 'Aaron Espe',
 'Tyler Ward',
 'Five For Fighting',
 'Bailey Jehl',
 'Nusrat Fateh Ali Khan',
 'Drew Holcomb & The Neighbors',
 'Gabrielle Aplin',
 'The Civil Wars',
 'Callum J Wright',
 'Fifth Harmony',
 'JP Cooper',
 'Mone Kamishiraishi',
 'John Adams',
 'Kitri',
 'Augustana',
 'Matthew Perryman Jones',
 'Ray LaMontagne',
 'Sierra Ferrell',
 'Meg Birch',
 'Catherine Feeny',
 

In [ ]:
# Output format
# A list/pandas dataframe where each element is a song entry in the recommended playlist
# Cols: 'track_index', 'track_id', 'artists', 'album_name', 'track_name', 'duration_ms', 'explicit', 'danceability', 'energy',
      #  'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
      #  'track_genre'
# [[]]

In [ ]:
# Input format
# A list where each element is a tuple representing the data for a certain group of attributes. These groups are: [artists, genres, audio_features, key, mode]
# The tuple would be of the form (the_attribute_values, weight)
# The weight is optional, representing how much each value would be considered - must be a float between 0 and 1 OR -1 (default) if the user doesn't choose a weight
  # for artists, the_attribute_values should be a list of all artist names the user has selected (the list may have 1 or multiple elements). The artist names must be the valid string format as found in the dataset - see the unique_artists variable two cells above
  # for genres,  the_attribute_values should be a list of all genres the user has selected (must be valid names from dataset). The list may have 1 or multiple elements
  # for audio_features, this is a list of values for these audio features in this order: 'duration_ms', 'explicit', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'
    # most of these values have to be floats between 0 and 1, but explicit has to be True or False
  # for key, just 1 value (not a list) - an int between 0 and 11 inclusive. See the graph on right hand for reference: https://en.wikipedia.org/wiki/Pitch_class#Other_ways_to_label_pitch_classes
  # for mode, just 1 value (not a list) - either 0 or 1 - Major is represented by 1 and minor is 0.

# [(artists_data, weight), (genres_data, weight), (audio_features_data, weight), (key_data, weight), (mode_data, weight)]

# Potentially useful reference about different audio features: https://developer.spotify.com/documentation/web-api/reference/get-audio-features

In [ ]:
# New: ALTERNATIVE INPUT FORMAT
# Please let me know which one of the input formats you'd be giving me.
# [(artists_data, weight), (genres_data, weight), (audio_feature1_explicit_data, weight), (audio_feature2_duration_ms_data, weight),
#  (audio_feature3_danceability_data, weight), ..., (audio_feature12_time_signature_data, weight), (key_data, weight), (mode_data, weight)]

# In this version, each of the audio features would have a separate tuple for its data(scalar between 0 and 1) and its own individual weight
# audio_features_in_order = ['explicit', 'duration_ms', 'danceability', 'energy',
                  # 'loudness', 'speechiness', 'acousticness',
                  # 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']

In [ ]:
# see this for datatypes reference
df.dtypes

track_index           int64
track_id             object
artists              object
album_name           object
track_name           object
duration_ms         float64
explicit               bool
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature      float64
track_genre          object
dtype: object

# ***Combining Everything***

In [ ]:
# Add all the engineered features in one place - TODO: define function above that would do this but + weights
# final = pd.concat([artists_df, genre_df, audio_scaled, one_hot_key, one_hot_mode], axis = 1)
# final.head()

In [ ]:
# Same code as the previous cell but in chuncks so it wouldn't overwork the RAM
# update: this is still too big
# final = artists_df
# final = pd.concat([final, genre_df], axis = 1)
# final = pd.concat([final, audio_scaled], axis = 1)
# final = pd.concat([final, one_hot_key], axis = 1)
# final = pd.concat([final, one_hot_mode], axis = 1)

In [ ]:
# Nevermind, this isn't a thing :(
# Same code but this time using append. It would take more time but hopefully less memory
# final = artists_df
# final.append(genre_df, inplace=True)
# final.append(audio_scaled, inplace=True)
# final.append(one_hot_key, inplace=True)
# final.append(one_hot_mode, inplace=True)

In [ ]:
# For now, to avoid runtime issues, let's do this without artists_df
final = pd.concat([genre_df, audio_scaled, one_hot_key, one_hot_mode], axis = 1)

NameError: name 'genre_df' is not defined

In [ ]:
final.head()

,genre|acoustic,genre|afrobeat,genre|age,genre|alt,genre|alternative,genre|ambient,genre|and,genre|anime,genre|bass,genre|black,...,key|4,key|5,key|6,key|7,key|8,key|9,key|10,key|11,mode|0,mode|1
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
# Add track ID's / track index
# final['id'] = df['track_index'].values
final.insert(0, 'id', df['track_index'].values)   # This does the same thing but adds it at the beginning

In [ ]:
final.head()

,id,genre|acoustic,genre|afrobeat,genre|age,genre|alt,genre|alternative,genre|ambient,genre|and,genre|anime,genre|bass,...,key|4,key|5,key|6,key|7,key|8,key|9,key|10,key|11,mode|0,mode|1
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
2,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
3,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
4,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True


# Cosine Similarity Function

In [ ]:
# see below

# ***Combining Everything: FUNCTION***

Includes 3 Important functions

- get_final_attributes: Everything we've done to this point (excluding Cosine Similarity section).
- get_ghost_song: make a ghost song using the attributes selected by the user
- use that ghost song to generate recommendations


More details:
- get_final_attributes: This function would do everything we've done in TFIDF Vectorization, one-hot encoding, normalization, and combining everything sections.
It will not take artists into consideration
Returns final (similar to final generated in the *Combining Everything* section)

Other functions here:
- get_final_attributes_and_artists: in progress (hopefully possible). Same as get_final_attributes but final will also include artists
- col_ohe: this is brought from one hot encoding section
- custom_tokenizer: brought from TFIDF section for artists
- todo: update_weights(final_df) --> weighted_final_df

In [ ]:
# run this line if you haven't already done so in the one-hot encoding section
def col_ohe(df, column):
  term_freq_col = pd.get_dummies(df[column])
  feature_names = term_freq_col.columns
  term_freq_col.columns = [column + "|" + str(i) for i in feature_names]
  term_freq_col.reset_index(drop=True, inplace=True)
  return term_freq_col

In [ ]:
# run this line if you haven't already done so in TFIDF section
def custom_tokenizer(text):   # used for artists only
  return text.split(';')

In [ ]:
# def get_final_attributes(df)
def get_final_attributes(df):
  # initial housekeeping
  df.rename(columns={'Unnamed: 0': 'track_index'}, inplace=True)
  df = df.dropna()
  df = df.drop('popularity', axis = 1)

  # TFIDF for genres
  df['track_genre'] = df['track_genre'].apply(lambda x: [x])  # adds a []
  tfidf = TfidfVectorizer()
  genres_tfidf_matrix = tfidf.fit_transform(df['track_genre'].apply(lambda x: " ".join(x)))
  genre_df = pd.DataFrame(genres_tfidf_matrix.toarray())
  genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]   # this makes the column heads reflect the genres that they represent
  genre_df.reset_index(drop = True, inplace=True)

  # one hot encoding for key and mode
  one_hot_mode = col_ohe(df, 'mode')
  one_hot_key = col_ohe(df, 'key')

  # scaling all attributes
  numerical_features = ['explicit', 'duration_ms', 'danceability', 'energy',
                        'loudness', 'speechiness', 'acousticness', 'instrumentalness',
                        'liveness', 'valence', 'tempo', 'time_signature']

  scaler = MinMaxScaler()
  audio = df[numerical_features].reset_index(drop= True)
  audio_scaled = pd.DataFrame(scaler.fit_transform(audio), columns = audio.columns)

  final = pd.concat([genre_df, audio_scaled, one_hot_key, one_hot_mode], axis = 1)
  return final

In [ ]:
# # def get_final_attributes_and_artists(df) (including artists)
# # This is a test that might or might not work
# def get_final_attributes_and_artists(df):
#   # initial housekeeping
#   df.rename(columns={'Unnamed: 0': 'track_index'}, inplace=True)
#   df = df.dropna()
#   df = df.drop('popularity', axis = 1)

#   # TFIDF for genres
#   df['track_genre'] = df['track_genre'].apply(lambda x: [x])  # adds a []
#   tfidf = TfidfVectorizer()
#   genres_tfidf_matrix = tfidf.fit_transform(df['track_genre'].apply(lambda x: " ".join(x)))
#   genre_df = pd.DataFrame(genres_tfidf_matrix.toarray())
#   genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]   # this makes the column heads reflect the genres that they represent
#   genre_df.reset_index(drop = True, inplace=True)

#   # one hot encoding for key and mode
#   one_hot_mode = col_ohe(df, 'mode')
#   one_hot_key = col_ohe(df, 'key')

#   # scaling all attributes
#   numerical_features = ['explicit', 'duration_ms', 'danceability', 'energy',
#                         'loudness', 'speechiness', 'acousticness', 'instrumentalness',
#                         'liveness', 'valence', 'tempo', 'time_signature']

#   scaler = MinMaxScaler()
#   audio = df[numerical_features].reset_index(drop= True)
#   audio_scaled = pd.DataFrame(scaler.fit_transform(audio), columns = audio.columns)

#   # TFIDF for artists
#   custom_tfidf = TfidfVectorizer(tokenizer=custom_tokenizer)
#   artists_tfidf_matrix = custom_tfidf.fit_transform(df['artists'])
#   artists_df = pd.DataFrame(artists_tfidf_matrix.toarray())
#   artists_df.columns = ['artist' + "|" + i for i in custom_tfidf.get_feature_names_out()]
#   artists_df.reset_index(drop = True, inplace=True)

#   # combine everything


In [ ]:
# def get_final_attributes_and_artists(df) (including artists)
# This is a test that might or might not work  ---> It works!!!
def get_final_attributes_and_artists(df):
  # Get everything else but artists
  semifinal = get_final_attributes(df)
  df = df.dropna()

  # TFIDF for artists
  custom_tfidf = TfidfVectorizer(tokenizer=custom_tokenizer)
  artists_tfidf_matrix = custom_tfidf.fit_transform(df['artists'])
  artists_df = pd.DataFrame(artists_tfidf_matrix.toarray())
  artists_df.columns = ['artist' + "|" + i for i in custom_tfidf.get_feature_names_out()]
  artists_df.reset_index(drop = True, inplace=True)

  # combine everything column by column. Insert semifinal to the end of artists_df
  for column_name, column_data in semifinal.items():
    artists_df.insert(len(artists_df.columns), column_name, column_data)

  return artists_df
  # combine everything - ... In Chunks. Code this part if the previous for loop still takes up RAM
  # start with artists_df. Insert a bit of semifinal to the end of artists_df
  # chunk_size =
  # for i in range(0, semifinal.shape[1], chunk_size){}

In [ ]:
# def get_final_attributes_and_artists(df) (including artists)
# This is a test that might or might not work  ---> It works!!!
def get_final_attributes_and_artists_weighted(df, user_input):
  # New: some setup
  song_attributes = [tpl[0] for tpl in user_input]  #using input v2
  song_weights = [tpl[1] for tpl in user_input]  #using input v2

  audio_features_in_order = ['explicit', 'duration_ms', 'danceability', 'energy',
                             'loudness', 'speechiness', 'acousticness', 'instrumentalness',
                             'liveness', 'valence', 'tempo', 'time_signature']

  # Get everything else but artists
  semifinal = get_final_attributes(df)
  df = df.dropna()

  # New: add weights to semifinal
  add_weights_final_df_artists(semifinal, user_input)

  # TFIDF for artists
  custom_tfidf = TfidfVectorizer(tokenizer=custom_tokenizer)
  artists_tfidf_matrix = custom_tfidf.fit_transform(df['artists'])
  artists_df = pd.DataFrame(artists_tfidf_matrix.toarray())
  artists_df.columns = ['artist' + "|" + i for i in custom_tfidf.get_feature_names_out()]
  artists_df.reset_index(drop = True, inplace=True)

  # New: add artists weights
  artists_df = artists_df * song_weights[0]

  # combine everything column by column. Insert semifinal to the end of artists_df
  for column_name, column_data in semifinal.items():
    artists_df.insert(len(artists_df.columns), column_name, column_data)

  return artists_df
  # combine everything - ... In Chunks. Code this part if the previous for loop still takes up RAM
  # start with artists_df. Insert a bit of semifinal to the end of artists_df
  # chunk_size =
  # for i in range(0, semifinal.shape[1], chunk_size){}

In [ ]:
# art_final = get_final_attributes_and_artists(df)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
<ipython-input-16-8fd01a8f26e3>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  artists_df.insert(len(artists_df.columns), column_name, column_data)
<ipython-input-16-8fd01a8f26e3>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  artists_df.insert(len(artists_df.columns), column_name, column_data)
<ipython-input-16-8fd01a8f26e3>:17: Performa

In [ ]:
# art_final.head()    # Yesss!!!

,artist|!nvite,"artist|""cats"" 1983 broadway cast","artist|""little"" jimmie dickens","artist|""pepo"" san martín","artist|""puppy dog pals"" cast","artist|""sax"" maldita vecindad","artist|""the phantom of the opera"" 1990 german cast","artist|""weird al"" yankovic",artist|#kids,artist|$affie,...,key|4,key|5,key|6,key|7,key|8,key|9,key|10,key|11,mode|0,mode|1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,True,False
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
# Function that adds the weights to final dataframe. The version without artists.
# final would be the output of get_final_attributes
# def add_weights_final_df(final, user_input):
  # Columns order: genre_df, audio_scaled, one_hot_key, one_hot_mode


In [ ]:
# Function that adds the weights to final dataframe. The version with artists.
# final would be the output of get_final_attributes_and_artists
def add_weights_final_df_artists(final, user_input):
  song_attributes = [tpl[0] for tpl in user_input]  #using input v2
  song_weights = [tpl[1] for tpl in user_input]  #using input v2

  audio_features_in_order = ['explicit', 'duration_ms', 'danceability', 'energy',
                             'loudness', 'speechiness', 'acousticness', 'instrumentalness',
                             'liveness', 'valence', 'tempo', 'time_signature']


  # for every col ... multiply by weight
  for i in range(0, final.shape[1]):
    col_name = final.columns[i]
    split_col_name = col_name.split("|")

    # artists
    if split_col_name[0] == 'artist':
      final[col_name] = final[col_name] * song_weights[0] # artists weight

    # genres
    elif split_col_name[0] == 'genre':
      final[col_name] = final[col_name] * song_weights[1] # genre weight

    # audio_features (scalars)
    elif split_col_name[0] in audio_features_in_order:
      # get matching index in audio features
      # update based on that value
      attribute_index = audio_features_in_order.index(split_col_name[0]) # index 0 == explicit.
      target_index = attribute_index + 2 # index 2 == explicit.
      final[col_name] = final[col_name] * song_weights[target_index]  # attribute weight

    # key
    elif split_col_name[0] == 'key':
      final[col_name] = final[col_name] * song_weights[-2] # key weight

    # mode
    elif split_col_name[0] == 'mode':
      final[col_name] = final[col_name] * song_weights[-1] # mode weight



In [ ]:
# def get_ghost_song(user_input)
# also updates weight
# assuming the second input format

# [(artists_data, weight), (genres_data, weight), (audio_feature1_explicit_data, weight), (audio_feature2_duration_ms_data, weight),
#  (audio_feature3_danceability_data, weight), ..., (audio_feature12_time_signature_data, weight), (key_data, weight), (mode_data, weight)]

# In this version, each of the audio features would have a separate tuple for its data(scalar between 0 and 1) and its own individual weight

# note for later: set final_columns_list = final.columns
# original col_list: original df.columns
def get_ghost_song(user_input, columns_list_i, columns_list_f):
  audio_features_in_order = ['explicit', 'duration_ms', 'danceability', 'energy',
                             'loudness', 'speechiness', 'acousticness', 'instrumentalness',
                             'liveness', 'valence', 'tempo', 'time_signature']

  song_attributes = ['artists', 'track_genre'] + audio_features_in_order + ['key', 'mode']

  song_values = [tpl[0] for tpl in user_input]  #using input v2
  song_weights = [tpl[1] for tpl in user_input]  #using input v2

  # Add weights to the attributes- todo: potentially remove or edit this part based on feedback
  # assuming that the weights are all scaled btw 0 and 1
  # for i in range(0, len(song_values)):
  #   song_values[i] = song_values[i] * song_weights[i]

  # convert this list into a dataframe with the same col titles as original df
  song_df = pd.DataFrame([song_values], columns = columns_list_i)
  # # create new empty df
  # new_song_df = pd.DataFrame(columns = columns_list_f)
  # # add a row of all zeros
  # zero_row =[0] * len(columns_list_f)
  # # new_song_df = df.append(pd.DataFrame([zero_row], columns= columns_list_f, ignore_index=True))
  # new_song_df = pd.concat([df, pd.DataFrame([zero_row], columns= columns_list_f)], ignore_index=True)  # the append method no longer in new pandas version

  # same thing as the commented bit above
  data = {col: [0.0] for col in columns_list_f}
  new_song_df = pd.DataFrame(data)

  # print stuff
  # print(song_df['artists'])
  # print(song_df['track_genre'])

  # This part replaced with the uncommented section below
  # ---------------------------------------------------
  # update the row values based on the song
  # new_song_artists = song_df['artists'].iloc[0].split(";")
  # new_song_genres = song_df['track_genre'].iloc[0].split(";")
  # for col_name, value in new_song_df.iloc[0].items():
  #   split_col_name = col_name.split("|")
  #   # artists
  #   if split_col_name[0] == 'artist':
  #     value = split_col_name[1] in new_song_artists # boolean artist
  #     value = value * song_weights[0]

  #   # genres
  #   elif split_col_name[0] == 'genre':
  #     value = split_col_name[1] in new_song_genres # boolean genre
  #     value = value * song_weights[1]

  #   # audio_features (scalars)
  #   elif split_col_name[0] in audio_features_in_order:
  #     # get matching index in audio features
  #     # update based on that value
  #     # target_index = audio_features_in_order.index(split_col_name[0]) # index 0 == explicit.
  #     value = song_df[split_col_name[0]]

  #     attribute_index = audio_features_in_order.index(split_col_name[0]) # index 0 == explicit.
  #     target_index = attribute_index + 2 # index 2 == explicit.
  #     value = value * song_weights[target_index]  # attribute weight

  #   # key
  #   elif split_col_name[0] == 'key':
  #     value = split_col_name[0] == song_df['key']
  #     value = value * song_weights[-2]

  #   # mode
  #   elif split_col_name[0] == 'mode':
  #     value = split_col_name[0] == song_df['mode']
  #     value = value * song_weights[-1]

  # Turns that this doesn't actually modify value

  # ----------------------------------------------
  # new_song_artists = song_df['artists'].iloc[0].split(";")
  # new_song_genres = song_df['track_genre'].iloc[0].split(";")
  # for col_name, value in new_song_df.iloc[0].items():
  #   split_col_name = col_name.split("|")
  #   # artists
  #   if split_col_name[0] == 'artist':
  #     df[col_name] = (split_col_name[1] in new_song_artists) * song_weights[0]

  #   # genres
  #   elif split_col_name[0] == 'genre':
  #     df[col_name] = (split_col_name[1] in new_song_genres) * song_weights[1]

  #   # audio_features (scalars)
  #   elif split_col_name[0] in audio_features_in_order:
  #     attribute_index = audio_features_in_order.index(split_col_name[0]) # index 0 == explicit.
  #     target_index = attribute_index + 2 # index 2 == explicit.

  #     df[col_name] = song_df[split_col_name[0]] * song_weights[target_index]

  #   # key
  #   elif split_col_name[0] == 'key':
  #     df[col_name] = (split_col_name[0] == song_df['key']) * song_weights[-2]

    # # mode
    # elif split_col_name[0] == 'mode':
    #   df[col_name] = (split_col_name[0] == song_df['mode']) * song_weights[-1]
# -------------------------------------------------------------
  new_song_artists = song_df['artists'].iloc[0].split(";")
  new_song_genres = song_df['track_genre'].iloc[0].split(";")

  # replace artists and genres with the lowercase versions
  new_song_artists = [x.lower() for x in new_song_artists]
  new_song_genres = [x.lower() for x in new_song_genres]

  for col in new_song_df:
    # col is the same as col_name in previous versions
    split_col_name = col.split("|")
    # artists
    if split_col_name[0] == 'artist':
      new_song_df.at[0, col] = (split_col_name[1] in new_song_artists) * song_weights[0]
    # genres
    elif split_col_name[0] == 'genre':
      new_song_df.at[0, col] = (split_col_name[1] in new_song_genres) * song_weights[1]

    # audio_features (scalars)
    elif split_col_name[0] in audio_features_in_order:
      attribute_index = audio_features_in_order.index(col) # index 0 == explicit.
      target_index = attribute_index + 2 # index 2 == explicit.

      new_song_df.at[0, col] = song_df[col] * song_weights[target_index]

    # key
    elif split_col_name[0] == 'key':
      new_song_df.at[0, col] = (int(split_col_name[1]) == song_df['key']) * song_weights[-2]

    # mode
    elif split_col_name[0] == 'mode':
      new_song_df.at[0, col] = (int(split_col_name[1]) == song_df['mode']) * song_weights[-1]
# -----------------------------------------------------------------
  # version using apply()
  # new_song_df = new_song_df.apply(ghost_helper(col_name, song_df, song_weights), axis = 1)
  # new_song_df = new_song_df.apply(lambda x: ghost_helper(x.name[1], song_df, song_weights), axis=1)
  return new_song_df

In [ ]:
# # ghost helper
# def ghost_helper(col_name, song_df, song_weights):
#   # this function will be applied to the new_song_df in ghost
#   split_col_name = col_name.split("|")
#   # artists
#   if split_col_name[0] == 'artist':
#     new_song_artists = song_df['artists'].iloc[0].split(";")
#     return (split_col_name[1] in new_song_artists) * song_weights[0]
#   # genres
#   elif split_col_name[0] == 'genre':
#     new_song_genres = song_df['track_genre'].iloc[0].split(";")
#     return (split_col_name[1] in new_song_genres) * song_weights[1]

#   # audio_features (scalars)
#   elif split_col_name[0] in audio_features_in_order:
#     attribute_index = audio_features_in_order.index(split_col_name[0]) # index 0 == explicit.
#     target_index = attribute_index + 2 # index 2 == explicit.

#     return song_df[split_col_name[0]] * song_weights[target_index]

#   # key
#   elif split_col_name[0] == 'key':
#     return (split_col_name[1] == song_df['key']) * song_weights[-2]

#   # mode
#   elif split_col_name[0] == 'mode':
#     return (split_col_name[1] == song_df['mode']) * song_weights[-1]


In [ ]:
# art_final.shape

(113999, 29952)

In [ ]:
# def generate_recommendations()
def generate_recommendations(df, user_df, num_recs):
  # using cosine similarity function - num recs is the number of recommendations
  # Note: this would mutate df (add/edit a similarity column)
  df['similarity'] = cosine_similarity(df.values, user_df.values)[:, 0]
  top_recs = df.sort_values('similarity', ascending = False)
  return top_recs.head(num_recs)


In [ ]:
# combining everythnig
# have the dataset uploaded and converted to df ahead of time
def combining_everything(df, user_input, num_recs):
  # get the final dataframe
  # final = get_final_attributes_and_artists(df)
  # add_weights_final_df_artists(final, user_input)
  final = get_final_attributes_and_artists_weighted(df, user_input)

  # final list of columns
  audio_features_in_order = ['explicit', 'duration_ms', 'danceability', 'energy',
                             'loudness', 'speechiness', 'acousticness', 'instrumentalness',
                             'liveness', 'valence', 'tempo', 'time_signature']

  columns_list_i = ['artists', 'track_genre'] + audio_features_in_order + ['key', 'mode']
  columns_list_f = final.columns
  # get user song df
  new_song_df = get_ghost_song(user_input, columns_list_i, columns_list_f)

  # generate the recommendation
  return generate_recommendations(df, new_song_df, num_recs)

# Looking at some examples
If this section runs, the program works!

In [ ]:
# Install specific libraries not included in Colab by default

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('/content/dataset.csv')

In [ ]:
# Optional: see how the model behaves and recommends with some examples
sample_input = [("The Weeknd", 0.3), ("pop; techno", 0.2), (False, 0), (0.7, 0), (0.8, 0.1), (0.65, 0.1), (0.5, 0), (0.5, 0), (0.1, 0.1), (0, 0), (0.3, 0.05), (0.4, 0.05), (0.6, 0.04), (0, 0), (1, 0), (1, 0.06)]

In [ ]:
recs_df = combining_everything(df, sample_input, 10)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# Example 2

In [ ]:
# Install specific libraries not included in Colab by default

# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('/content/dataset.csv')

In [ ]:
user_input = [("The Weeknd", 0.3), ("pop;techno", 0.2), (False, 0), (0.7, 0), (0.8, 0.1), (0.65, 0.1), (0.5, 0), (0.5, 0), (0.1, 0.1), (0, 0), (0.3, 0.05), (0.4, 0.05), (0.6, 0.04), (0, 0), (1, 0), (1, 0.06)]

In [ ]:
user_input

[('The Weeknd', 0.3),
 ('pop;techno', 0.2),
 (False, 0),
 (0.7, 0),
 (0.8, 0.1),
 (0.65, 0.1),
 (0.5, 0),
 (0.5, 0),
 (0.1, 0.1),
 (0, 0),
 (0.3, 0.05),
 (0.4, 0.05),
 (0.6, 0.04),
 (0, 0),
 (1, 0),
 (1, 0.06)]

In [ ]:
# get the final dataframe
final = get_final_attributes_and_artists(df)
# add_weights_final_df_artists(final, user_input)
# final = get_final_attributes_and_artists_weighted(df, user_input)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
<ipython-input-6-187f1e5532d6>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  artists_df.insert(len(artists_df.columns), column_name, column_data)
<ipython-input-6-187f1e5532d6>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  artists_df.insert(len(artists_df.columns), column_name, column_data)
<ipython-input-6-187f1e5532d6>:17: Performance

In [ ]:
# final list of columns
# columns_list_i = df.columns
# columns_list_i = [col for col in df.columns if col in {'artists', 'duration_ms', 'explicit', 'danceability', 'energy',
#        'key', 'loudness', 'mode', 'speechiness', 'acousticness','instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre'}]
columns_list_f = final.columns

In [ ]:
audio_features_in_order = ['explicit', 'duration_ms', 'danceability', 'energy',
                             'loudness', 'speechiness', 'acousticness', 'instrumentalness',
                             'liveness', 'valence', 'tempo', 'time_signature']

columns_list_i = ['artists', 'track_genre'] + audio_features_in_order + ['key', 'mode']

In [ ]:
columns_list_i

['artists',
 'track_genre',
 'explicit',
 'duration_ms',
 'danceability',
 'energy',
 'loudness',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'time_signature',
 'key',
 'mode']

In [ ]:
[x for x in columns_list_f if x in {'track_index', 'track_id', 'album_name', 'track_name'}]

[]

In [ ]:
[col for col in columns_list_i if col not in columns_list_f]

['artists', 'track_genre', 'key', 'mode']

In [ ]:
[col for col in columns_list_i if (col not in columns_list_f and col in audio_features_in_order)]

[]

In [ ]:
# get user song df
new_song_df = get_ghost_song(user_input, columns_list_i, columns_list_f)

<ipython-input-56-edc54a2fcdf0>:134: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  new_song_df.at[0, col] = song_df[col] * song_weights[target_index]
<ipython-input-56-edc54a2fcdf0>:134: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  new_song_df.at[0, col] = song_df[col] * song_weights[target_index]
<ipython-input-56-edc54a2fcdf0>:134: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  new_song_df.at[0, col] = song_df[col] * song_weights[target_index]
<ipython-input-56-edc54a2fcdf0>:138: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  new_song_df.at[0, col] = (int(split_col_name[1]) == song_df['key']) * s

In [ ]:
new_song_df

,artist|!nvite,"artist|""cats"" 1983 broadway cast","artist|""little"" jimmie dickens","artist|""pepo"" san martín","artist|""puppy dog pals"" cast","artist|""sax"" maldita vecindad","artist|""the phantom of the opera"" 1990 german cast","artist|""weird al"" yankovic",artist|#kids,artist|$affie,...,key|4,key|5,key|6,key|7,key|8,key|9,key|10,key|11,mode|0,mode|1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.06


In [ ]:
new_song_df['valence']

0    0.02
Name: valence, dtype: float64

In [ ]:
# # Ignore This
# for col, val in new_song_df.iloc[0].items():
#   # print(col, end=' ')
#   # print(val)
#   split_col_name = col.split("|")
#   # print(split_col_name[0])
#   # artists
#   if split_col_name[0] == "artist":
#     val = 1.0
#     print(split_col_name[0], end=" ")
#     print(val)
#   #   val = split_col_name[1] in new_song_artists # boolean artist
#   #   val = value * song_weights[0]

In [ ]:
new_song_df["artist|the weeknd"]

0    0.3
Name: artist|the weeknd, dtype: float64

In [ ]:
new_song_df["genre|pop"]

0    0.2
Name: genre|pop, dtype: float64

In [ ]:
new_song_df["genre|techno"]

0    0.2
Name: genre|techno, dtype: float64

In [ ]:
# new_song_artists = user_input[0][0].lower().split(";")
# new_song_genres = user_input[1][0].lower().split(";")

# audio_features_in_order = ['explicit', 'duration_ms', 'danceability', 'energy',
#                              'loudness', 'speechiness', 'acousticness', 'instrumentalness',
#                              'liveness', 'valence', 'tempo', 'time_signature']

# song_attributes = ['artists', 'track_genre'] + audio_features_in_order + ['key', 'mode']

# song_values = [tpl[0] for tpl in user_input]  #using input v2
# song_weights = [tpl[1] for tpl in user_input]  #using input v2

# song_df = pd.DataFrame([song_values], columns = columns_list_i)

# for col in new_song_df:
#   # col is the same as col_name in previous versions
#   split_col_name = col.split("|")
#   # artists
#   if split_col_name[0] == 'artist':
#     new_song_df.at[0, col] = (split_col_name[1] in new_song_artists) * song_weights[0]
#   # genres
#   elif split_col_name[0] == 'genre':
#     new_song_df.at[0, col] = (split_col_name[1] in new_song_genres) * song_weights[1]

#   # audio_features (scalars)
#   elif split_col_name[0] in audio_features_in_order:
#     attribute_index = audio_features_in_order.index(col) # index 0 == explicit.
#     target_index = attribute_index + 2 # index 2 == explicit.

#     new_song_df.at[0, col] = song_df[col] * song_weights[target_index]

#   # key
#   elif split_col_name[0] == 'key':
#     new_song_df.at[0, col] = (split_col_name[1] == song_df['key']) * song_weights[-2]

#   # mode
#   elif split_col_name[0] == 'mode':
#     new_song_df.at[0, col] = (split_col_name[1] == song_df['mode']) * song_weights[-1]

<ipython-input-50-05108f79cbdf>:30: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  new_song_df.at[0, col] = song_df[col] * song_weights[target_index]
<ipython-input-50-05108f79cbdf>:30: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  new_song_df.at[0, col] = song_df[col] * song_weights[target_index]
<ipython-input-50-05108f79cbdf>:30: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  new_song_df.at[0, col] = song_df[col] * song_weights[target_index]
<ipython-input-50-05108f79cbdf>:34: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  new_song_df.at[0, col] = (split_col_name[1] == song_df['key']) * song_weigh

In [ ]:
# generate the recommendation
recs = generate_recommendations(final, new_song_df, 10)

# Save model for later use (optional)

In [ ]:
# Optional: put here code to save model for later use / backup
# you don't have to run this everytime and you might need to connect it to your google drive to save it

# Connect to frontend?

In [ ]:
# any code here that we might need to get this to the front end stuff, or get the front end stuff here